In [ ]:
from functions_needed import *

In [ ]:
!rm file_phasetran.txt

import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
from scipy.special import jn_zeros, j0
import math
from qutip import Options
from functions_needed import*
import h5py, time
freezing_pts = jn_zeros(0, 500)
start = time.time()

Ns = [200,2]
ostp = [1e7, 1e4]
frz_jnth = [5,5]

# which number of floquet mode we take
nth = 0           
# number of freezing point
#frz_nth = 20
h0 = np.sqrt(3)/2**4
ws = np.exp(np.linspace(np.log(.1), np.log(100), 40))

ws1 = np.exp(np.linspace(np.log(0.01), np.log(0.1), 4))
ws2 = np.exp(np.linspace(np.log(0.2), np.log(0.9), 10))
ws3 = np.exp(np.linspace(np.log(1.0), np.log(30.0), 20))
ws4 = np.exp(np.linspace(np.log(35.0), np.log(200), 5))
#ws = np.concatenate((ws1,ws2,ws3,ws4))

if __name__ == '__main__':
    nprocs = 8
    #opts = Options(nsteps=1e4)    
    
    for nn,N in enumerate(Ns):
        start1 = time.time()
        opts = Options(nsteps = ostp[nn])
        frz_nth = frz_jnth[nn] 
        params = [{'h0':h0, 'h': 1/4 * w * freezing_pts[frz_nth], 'omega':w, 'N':N, 'opts':opts} for w in ws]
        p = Pool(processes = nprocs)  

        data = np.array(p.map(floq_evolv_exact_sten,params))

        w_f = np.array(data[:,0])
        evec_ff = np.array(data[:,1])
        phs_f = np.array(data[:,2])

        evec_f = np.array([np.array([np.array(evec_ff[j][i])[:,0].real for i in range(int(2*N+1))]) for j in range(len(ws))])

        evals_single_path = np.zeros(len(ws))
        iprmat_single_path = np.zeros(len(ws))
        iprmat = np.zeros((len(ws),2*N+1))

        sx = jmat(N,"x")
        en, st = sx.eigenstates()

        for xx,w in enumerate(ws):
            #Get the previous Floquet Eigensystem
            if xx == 0:
                evals_prev = phs_f[xx].real
                evecs_prev = evec_f[xx]
            else:
                evals_prev = phs_f[xx-1].real
                evecs_prev = evec_f[xx-1]

            #Sort the Floquet states of this itertaion accordingly to orthonormality with the previous ones
            evals_curr = phs_f[xx]
            evecs_curr = evec_f[xx]
            t = np.copy(evecs_curr)
            condition = np.abs(np.around(evecs_prev.T.conj() @ evecs_curr, 1))
            evals_curr = evals_curr[np.argmax(condition, axis=1)]        
            evecs_curr = evecs_curr[:,np.argmax(condition, axis=1)]

            evals_single_path[xx] = evals_curr[nth]

            f_states = evec_f[xx]
            iprmat[xx,:] = [np.sum([np.abs(Qobj(state).overlap(sx_ev))**4 for sx_ev in st], axis=-1) for state in f_states]

            iprmat_single_path[xx] = iprmat[xx,:][nth]
               
        # save the data
        filename = 'phase_transition_11N' + str(N) +'_.hdf5'

        with h5py.File(filename, 'w') as hf:
            hf.create_dataset('ws', np.shape(ws), data=ws)
            hf.create_dataset('evals_single_path', np.shape(evals_single_path), data=evals_single_path)
            hf.create_dataset('iprmat_single_path', np.shape(iprmat_single_path), data=iprmat_single_path)
            hf.attrs['N'] = N
            hf.attrs['frz_nth'] = frz_nth
            hf.attrs['nth'] = nth
        
        print("For N = ", N, "with", nprocs, "processors : time taken ", (time.time()-start1)/60, ' minutes !')
    print('\n run complete!! \n\n TOTAL TIME TAKEN :', (time.time()-start)/60, ' minutes !')